In [20]:
from tableauscraper import TableauScraper as TS

url= "https://public.tableau.com/views/moph_covid_v3/Story1"
ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

#alias
wb = workbook

for i, t in enumerate(workbook.worksheets):
    print(f"{i} worksheet name : {t.name}") #show worksheet name
    print(t.data) #show dataframe for this worksheet


0 worksheet name : ppe_available
   SUM(Ppe Total)-alias
0                933351
1 worksheet name : respirator_available
   SUM(Ventilator)-alias
0                   9839
2 worksheet name : map_total
   Prov Name En-value Prov Name En-alias Latitude (generated)-value  \
0            Yasothon           Yasothon                    15.8189   
1                Yala               Yala                     6.1478   
2           Uttaradit          Uttaradit                    17.7591   
3         Uthai Thani        Uthai Thani                    15.3707   
4          Udon Thani         Udon Thani                    17.4469   
..                ...                ...                        ...   
72           Buri Ram           Buri Ram                    14.9638   
73          Bueng Kan          Bueng Kan                    18.1758   
74            Bangkok            Bangkok                    13.7249   
75          Ang Thong          Ang Thong                    14.6212   
76      Amnat Charo

In [6]:
wb.getParameters()

[]

In [3]:
province_ws = workbook.worksheets[9]
province_ws2 = workbook.getWorksheet("province_total")
province_ws is province_ws2

province_ws.data

,Prov Name-value,Prov Name-alias,Measure Names-alias,Measure Values-value,Measure Values-alias
0,อ่างทอง,อ่างทอง,เตียงใช้ไปแล้ว,0,0
1,สิงห์บุรี,สิงห์บุรี,เตียงใช้ไปแล้ว,4,4
2,สมุทรสาคร,สมุทรสาคร,เตียงใช้ไปแล้ว,19,19
3,แม่ฮ่องสอน,แม่ฮ่องสอน,เตียงใช้ไปแล้ว,49,49
4,ลำพูน,ลำพูน,เตียงใช้ไปแล้ว,63,63
...,...,...,...,...,...
149,ปัตตานี,ปัตตานี,เตียงพร้อมใช้,1054,"1,054"
150,นราธิวาส,นราธิวาส,เตียงพร้อมใช้,1298,"1,298"
151,กรุงเทพมหานคร,กรุงเทพมหานคร,เตียงพร้อมใช้,1474,"1,474"
152,สุรินทร์,สุรินทร์,เตียงพร้อมใช้,1660,"1,660"


In [5]:
filters = province_ws.getFilters(); filters

[]

In [4]:
selections = province_ws.getSelectableItems(); selections

[{'column': 'Prov Name',
  'values': ['อ่างทอง',
   'สิงห์บุรี',
   'สมุทรสาคร',
   'แม่ฮ่องสอน',
   'ลำพูน',
   'แพร่',
   'พังงา',
   'จันทบุรี',
   'ชัยนาท',
   'ตรัง',
   'สมุทรสงคราม',
   'นครปฐม',
   'ลพบุรี',
   'น่าน',
   'อุทัยธานี',
   'ระนอง',
   'นครนายก',
   'สระบุรี',
   'ภูเก็ต',
   'มุกดาหาร',
   'มหาสารคาม',
   'เชียงใหม่',
   'เลย',
   'ตาก',
   'อำนาจเจริญ',
   'พะเยา',
   'สมุทรปราการ',
   'สุโขทัย',
   'ปราจีนบุรี',
   'หนองคาย',
   'หนองบัวลำภู',
   'บึงกาฬ',
   'สตูล',
   'ยโสธร',
   'ลำปาง',
   'ราชบุรี',
   'สระแก้ว',
   'กำแพงเพชร',
   'เพชรบูรณ์',
   'ร้อยเอ็ด',
   'ปทุมธานี',
   'สุพรรณบุรี',
   'พิษณุโลก',
   'ฉะเชิงเทรา',
   'ชุมพร',
   'พระนครศรีอยุธยา',
   'พิจิตร',
   'ตราด',
   'ขอนแก่น',
   'นนทบุรี',
   'กาฬสินธุ์',
   'ศรีสะเกษ',
   'อุตรดิตถ์',
   'พัทลุง',
   'นครสวรรค์',
   'เพชรบุรี',
   'กระบี่',
   'ยะลา',
   'ประจวบคีรีขันธ์',
   'นครพนม',
   'เชียงราย',
   'อุบลราชธานี',
   'สุราษฎร์ธานี',
   'นครศรีธรรมราช',
   'ชัยภูมิ',
   'ชลบุรี',
   'บ

In [30]:
prov_name = selections[0];
len(prov_name['values']), len(set(prov_name['values']))

(154, 77)

In [ ]:
province_ws.select("Prov Name", "สมุทรสาคร")

In [8]:
map_ws = wb.getWorksheet('map_total'); map_ws.data

,Prov Name En-value,Prov Name En-alias,Latitude (generated)-value,Latitude (generated)-alias,Longitude (generated)-value,Longitude (generated)-alias,AGG(bed_percentage)-alias
0,Yasothon,Yasothon,15.8189,15.82,104.2734,104.273,95.811518
1,Yala,Yala,6.1478,6.15,101.2366,101.237,77.617329
2,Uttaradit,Uttaradit,17.7591,17.76,100.4466,100.447,88.548057
3,Uthai Thani,Uthai Thani,15.3707,15.37,99.5595,99.560,88.77551
4,Udon Thani,Udon Thani,17.4469,17.45,102.8325,102.833,85.413745
...,...,...,...,...,...,...,...
72,Buri Ram,Buri Ram,14.9638,14.96,103.0636,103.064,94.814815
73,Bueng Kan,Bueng Kan,18.1758,18.18,103.6605,103.661,72.8
74,Bangkok,Bangkok,13.7249,13.72,100.6034,100.603,91.723709
75,Ang Thong,Ang Thong,14.6212,14.62,100.3511,100.351,%null%


In [9]:
map_ws.getFilters()

[]

In [11]:
selectables = map_ws.getSelectableItems(); selectables

[{'column': 'Prov Name En',
  'values': ['Yasothon',
   'Yala',
   'Uttaradit',
   'Uthai Thani',
   'Udon Thani',
   'Ubon Ratchathani',
   'Trat',
   'Trang',
   'Tak',
   'Surin',
   'Surat Thani',
   'Suphan Buri',
   'Sukhothai',
   'Songkhla',
   'Sing Buri',
   'Si Sa Ket',
   'Satun',
   'Saraburi',
   'Samut Songkhram',
   'Samut Sakhon',
   'Samut Prakan',
   'Sakon Nakhon',
   'Sa Kaeo',
   'Roi Et',
   'Rayong',
   'Ratchaburi',
   'Ranong',
   'Prachuap Khiri Khan',
   'Prachin Buri',
   'Phuket',
   'Phrae',
   'Phra Nakhon Si Ayutthaya',
   'Phitsanulok',
   'Phichit',
   'Phetchaburi',
   'Phetchabun',
   'Phayao',
   'Phatthalung',
   'Phangnga',
   'Pattani',
   'Pathum Thani',
   'Nonthaburi',
   'Nong Khai',
   'Nong Bua Lam Phu',
   'Narathiwat',
   'Nan',
   'Nakhon Si Thammarat',
   'Nakhon Sawan',
   'Nakhon Ratchasima',
   'Nakhon Phanom',
   'Nakhon Pathom',
   'Nakhon Nayok',
   'Mukdahan',
   'Maha Sarakham',
   'Mae Hong Son',
   'Lop Buri',
   'Loei',
   '

In [12]:
len(selectables[0]['values'])

77

In [14]:
map_ws.select('Prov Name En', 'Yala')

TypeError: 'NoneType' object is not subscriptable

In [21]:
wb.getStoryPoints()

{'storyBoard': 'Story 1',
 'storyPoints': [[{'storyPointId': 12, 'storyPointCaption': ' ทรัพยากรภาพรวม'},
   {'storyPointId': 1, 'storyPointCaption': '          เตียง'},
   {'storyPointId': 2, 'storyPointCaption': '    หน้ากาก N95 '},
   {'storyPointId': 3, 'storyPointCaption': '    หน้ากากอนามัย'},
   {'storyPointId': 4, 'storyPointCaption': 'ชุด PPE (Cover all)'},
   {'storyPointId': 5, 'storyPointCaption': '         NIV'},
   {'storyPointId': 6, 'storyPointCaption': '     VENTILATOR'}]]}

In [22]:
sp = wb.goToStoryPoint(storyPointId=12)

In [23]:
dir(sp)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_originalData',
 '_originalInfo',
 '_scraper',
 'cmdResponse',
 'getCrossTabData',
 'getCsvData',
 'getDownloadableData',
 'getParameters',
 'getSheets',
 'getStoryPoints',
 'getWorksheet',
 'getWorksheetNames',
 'getWorksheets',
 'goToSheet',
 'goToStoryPoint',
 'setParameter',
 'updateFullData',
 'worksheets']

In [24]:
sp.getParameters()

[]

In [4]:
print(sp.getWorksheetNames())

['bed_available', 'map_total', 'doughtnut_cohort', 'doughtnut_aiir', 'isolation', 'doughtnut_total', 'bed_total', 'modified_aiir', 'moph_bed', 'province_total']


In [26]:
for ws in sp.worksheets:
    print(ws.name)
    print(ws.data)

ppe_available
   SUM(Ppe Total)-alias
0                933351
respirator_available
   SUM(Ventilator)-alias
0                   9839
map_total
   Prov Name En-value Prov Name En-alias Latitude (generated)-value  \
0            Yasothon           Yasothon                    15.8189   
1                Yala               Yala                     6.1478   
2           Uttaradit          Uttaradit                    17.7591   
3         Uthai Thani        Uthai Thani                    15.3707   
4          Udon Thani         Udon Thani                    17.4469   
..                ...                ...                        ...   
72           Buri Ram           Buri Ram                    14.9638   
73          Bueng Kan          Bueng Kan                    18.1758   
74            Bangkok            Bangkok                    13.7249   
75          Ang Thong          Ang Thong                    14.6212   
76      Amnat Charoen      Amnat Charoen                    15.9103   

   L

In [31]:
occupation = sp.getWorksheet('province_total')

In [34]:
df = occupation.data[['Prov Name-value', 'Measure Names-alias', 'Measure Values-value']]

In [37]:
df.columns = ['Province', 'ValueType', 'Value']

In [39]:
df = df.pivot('Province', 'ValueType', 'Value')

In [42]:
df.columns = ['Availiable', 'Occupied']

In [43]:
df

,Availiable,Occupied
Province,,
กระบี่,303,239
กรุงเทพมหานคร,1474,133
กาญจนบุรี,952,221
กาฬสินธุ์,423,37
กำแพงเพชร,310,84
...,...,...
เพชรบุรี,496,44
เพชรบูรณ์,313,85
เลย,224,49


In [45]:
df.isna().any().any()

False